In [1]:
import pandas as pd
commands=pd.read_csv("TextCommands.csv")
commands.columns = ['text','label','misc']           
commands.head()

,text,label,misc
0,Undo the last sentence,1,NaN
1,Undo the last word,1,NaN
2,Can you undo the last sentence,1,NaN
3,Please undo the text,1,NaN
4,Undo the selected text,1,NaN


In [2]:
commands.label.value_counts()

6     26
18    23
14    22
13    21
24    21
17    19
2     19
1     18
4     17
3     17
15    15
5     15
26    15
16    14
19    14
12    13
21    13
25    13
20    12
9     12
8     11
7     10
10    10
22    10
11    10
23     9
Name: label, dtype: int64

Data preprocessing is a particularly important task in NLP. We apply three main pre-processing methods here :

Tokenizing: Keras’ inbuilt tokenizer API has fit the dataset which splits the sentences into words and creates a dictionary of all unique words found and their uniquely assigned integers. Each sentence is converted into an array of integers representing all the unique words present in it.

In [3]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_SEQUENCE_LENGTH = 10
MAX_NUM_WORDS = 5000

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(commands['text'])
sequences = tokenizer.texts_to_sequences(commands['text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(commands['label']))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 142 unique tokens.
Shape of data tensor: (399, 10)
Shape of label tensor: (399, 27)


In [4]:
import json

with open( 'word_dict.json' , 'w' ) as file:
	json.dump( tokenizer.word_index , file )

In [5]:
VALIDATION_SPLIT = 0.1
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [6]:
x_train.shape

(360, 10)

#A keras functional model is implemented. It has the following layers :
An input layer that takes the array of length 10 representing a sentence.
An embedding layer of dimension 60 whose weights can be updated during training. It helps to convert each word into a fixed-length dense vector of size 60. The input dimension is set as the size of the vocabulary and the output dimension is 60. Each word in the input will hence get represented by a vector of size 60.
Two convolutional layers (Conv1D) with 64 filters each, kernel size of 3, and relu activation.
A max-pooling layer(MaxPooling1D) with pool size 2. Max Pooling in CNN is an operation that selects the maximum element from the region of the input which is covered by the filter/kernel. Pooling reduces the dimensions of the output, but it retains the most important information.
A flatten layer to flatten the input without affecting batch size. If the input to the flatten layer is a tensor of shape 1 X 3 X 64, the output will be a tensor of shape 1 X 192.
A dense (fully connected) layer of 100 units and relu activation.
A dense layer of 26 units and softmax activation that outputs the final probabilities of belonging to each of the 26 classes. Softmax activation is used here since it goes best with categorical cross-entropy loss, which is the loss we are going to be using to train the model.


In [7]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.models import Sequential
from keras.initializers import Constant
EMBEDDING_DIM = 60
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_layer = Embedding(num_words,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH,trainable=True)


In [8]:
print('Training model.')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(64, 3, activation='relu')(embedded_sequences)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(2)(x)

x=Flatten()(x)
x = Dense(100, activation='relu')(x)
preds = Dense(27, activation='softmax')(x)

model = Model(sequence_input, preds)


Training model.


The model is compiled with categorical cross-entropy loss and rmsprop optimizer. Categorical cross-entropy is a loss function commonly used for multi-class classification tasks. The rmsprop optimizer is a gradient-based optimization technique that uses a moving average of squared gradients to normalize the gradient. This helps to overcome the vanishing gradients problem. Accuracy is used as the main performance metric. The model summary can be seen below :

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 60)            8580      
                                                                 
 conv1d (Conv1D)             (None, 8, 64)             11584     
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 64)             12352     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3, 64)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 192)               0         
                                                             

In [11]:
#Model Training and Evaluation
#The model is trained for 30 epochs with batch size 50.
s=0.0
for i in range (1,50):
    model.fit(x_train, y_train,batch_size=50, epochs=30, validation_data=(x_val, y_val))
    # evaluate the model
    scores = model.evaluate(x_val, y_val, verbose=0)
    s=s+(scores[1]*100)

Epoch 1/30
8/8 [==============================] - 1s 40ms/step - loss: 3.2762 - acc: 0.0694 - val_loss: 3.2840 - val_acc: 0.0000e+00
Epoch 2/30
8/8 [==============================] - 0s 9ms/step - loss: 3.1949 - acc: 0.0722 - val_loss: 3.2542 - val_acc: 0.0000e+00
Epoch 3/30
8/8 [==============================] - 0s 9ms/step - loss: 3.0792 - acc: 0.0722 - val_loss: 3.1698 - val_acc: 0.0000e+00
Epoch 4/30
8/8 [==============================] - 0s 9ms/step - loss: 2.9346 - acc: 0.1000 - val_loss: 3.0803 - val_acc: 0.1538
Epoch 5/30
8/8 [==============================] - 0s 11ms/step - loss: 2.7570 - acc: 0.1972 - val_loss: 3.0111 - val_acc: 0.1538
Epoch 6/30
8/8 [==============================] - 0s 9ms/step - loss: 2.5562 - acc: 0.2889 - val_loss: 2.8020 - val_acc: 0.2308
Epoch 7/30
8/8 [==============================] - 0s 12ms/step - loss: 2.3329 - acc: 0.3694 - val_loss: 2.5568 - val_acc: 0.3077
Epoch 8/30
8/8 [==============================] - 0s 11ms/step - loss: 2.1237 - acc: 0.42

In [12]:
# evaluate the model
scores = model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 100.00%


In [13]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


In [14]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 100.00%


In [15]:
#new instance where we do not know the answer
Xnew=["kindly undo the changes","Can you please undo the last paragraph","Make bold this","Would you be kind enough to bold the last word?","Please remove bold from the last paragraph","Kindly unbold the selected text","Kindly insert comment here","Can you please put a comment here","Can you please centre align this text","Can you please position this text in the middle"]

sequences_new = tokenizer.texts_to_sequences(Xnew)

data = pad_sequences(sequences_new, maxlen=MAX_SEQUENCE_LENGTH)

# make a prediction
yprob = model.predict(data)
yclasses=yprob.argmax(axis=-1)
# show the inputs and predicted outputs
print("X=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s\nX=%s, Predicted=%s" % (Xnew[0], yclasses[0],Xnew[1],yclasses[1],Xnew[2],yclasses[2],Xnew[3],yclasses[3],Xnew[4],yclasses[4],Xnew[5],yclasses[5],Xnew[6],yclasses[6],Xnew[7],yclasses[7],Xnew[8],yclasses[8],Xnew[9],yclasses[9]))

1/1 [==============================] - 0s 108ms/step
X=kindly undo the changes, Predicted=1
X=Can you please undo the last paragraph, Predicted=1
X=Make bold this, Predicted=2
X=Would you be kind enough to bold the last word?, Predicted=2
X=Please remove bold from the last paragraph, Predicted=3
X=Kindly unbold the selected text, Predicted=3
X=Kindly insert comment here, Predicted=15
X=Can you please put a comment here, Predicted=15
X=Can you please centre align this text, Predicted=14
X=Can you please position this text in the middle, Predicted=14


In [18]:
# Converting a saved keras model to a Tensorflow Lite model
#https://www.tensorflow.org/api_docs/python/tf/compat/v1/lite/TFLiteConverter
import tensorflow as tf
import keras
keras_file = "model.h5"
keras.models.save_model(model,keras_file)
converter =  tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(keras_file)
converter.post_training_quantize = True
tfmodel = converter.convert()
open("model.tflite","wb").write(tfmodel)

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/lite.py:1894: UserWarning: Property post_training_quantize is deprecated, please use optimizations=[Optimize.DEFAULT] instead.
  " instead." % name)


61888